<a href="https://colab.research.google.com/github/0seoyeon/tobigs-21st/blob/%EA%B3%B5%EC%84%9C%EC%97%B0/Week4/Ensemble/%EA%B3%B5%EC%84%9C%EC%97%B0/Tobigs_Week4_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tobigs 21기 4주차 정규세션 Ensemble

### 앙상블 모델 최소 2가지를 이용해 코드를 짜주세요!
- 전처리와 모델 성능결과 출력 모두 진행해주세요.
- 데이터셋: https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers
    - 다른 데이터셋을 사용해도 괜찮습니다! 대신 출처를 명시해주세요.

# Load Data

In [175]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [213]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import classification_report

In [177]:
import os

# 현재 작업 디렉토리 확인
current_directory = os.getcwd()
print("Current directory:", current_directory) # /content

# 작업 디렉토리 내의 파일 목록 확인
# os.listdir(current_directory)

# 작업 디렉토리 변경
new_directory = os.chdir('/content/drive/MyDrive/Tobigs/regular_session')
current_directory = os.getcwd()
print('Current directory:', current_directory)

Current directory: /content/drive/My Drive/Tobigs/regular_session
Current directory: /content/drive/MyDrive/Tobigs/regular_session


In [178]:
df = pd.read_csv('./data/BankChurners.csv')

# DataPrep

In [179]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 23 columns):
 #   Column                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                              --------------  -----  
 0   CLIENTNUM                                                                                                                           10127 non-null  int64  
 1   Attrition_Flag                                                                                                                      10127 non-null  object 
 2   Customer_Age                                                                                                                        10127 non-null  int64  
 3   Gender                                                                           

## BankChurners.csv Overview
Objective: predict customers who are likely to drop off.
- CLIENTNUM: Client number. Unique identifier for the customer holding the account  
- **Attrition_Flag: Internal event (customer activity) variable - if the account is closed then 1 else 0 -> Dependent variable**
- Customer_Age: Demographic variable - Customer's Age in Years
- Gender: Demographic variable - M=Male, F=Female  
- Dependent_count: Demographic variable - Number of dependents(피부양자)  
- Education_Level: Demographic variable - Educational Qualification of the account holder (example: high school, college graduate, etc.)
- Marital_Status: Demographic variable - Married, Single, Divorced, Unknown
- Income_Category: Demographic variable - Annual Income Category of the account holder (< $40K, $40K - 60K, $60K - $80K, $80K-$120K, > $120K, Unknown)
- Card_Category: Product Variable - Type of Card (Blue, Silver, Gold, Platinum)
- Months_on_book: Period of relationship with bank
- Total_Relationship_Count: Total no. of products held by the customer
- Months_Inactive_12_mon: No. of months inactive in the last 12 months
- Contacts_Count_12_mon: No. of Contacts in the last 12 months
- Credit_Limit: Credit Limit on the Credit Card
- Total_Revolving_Bal: Total Revolving Balance on the Credit Card (revolving: minimum payment로 정해진 돈만 갚으면 사용액이 다음 달로 넘어 가면서 이자가 붙는 방식이다. 따라서 revolving balance는 현재 사용한 금액)
- Avg_Open_To_Buy: Open to Buy Credit Line (Average of last 12 months) (이용 가능한 신용 한도의 평균적인 금액)
- Total_Amt_Chng_Q4_Q1: Change in Transaction Amount (Q4 over Q1)
- Total_Trans_Amt: Total Transaction Amount (Last 12 months)
- Total_Trans_Ct: Total Transaction Count (Last 12 months)
- Total_Ct_Chng_Q4_Q1: Change in Transaction Count (Q4 over Q1)
- Avg_Utilization_Ratio: Average Card Utilization Ratio
- Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1: 다른 변수들을 이용하여 나이브 베이즈 분류(0 또는 1) 적용한 결과
- Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2: 다른 변수들을 이용하여 나이브 베이즈 분류(0 또는 1) 적용한 결과

**PLEASE IGNORE THE LAST 2 COLUMNS (NAIVE BAYES CLAS…). I SUGGEST TO RATHER DELETE IT BEFORE DOING ANYTHING****

In [181]:
len(df['CLIENTNUM'].unique())

10127

In [182]:
# drop columns
df = df.drop(columns = ['CLIENTNUM',
                        'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                        'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'])
df.info() # no missing data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  object 
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

In [183]:
# distinguish Categorical featues and Numerical features
cat_col = [x for x in df.columns if df[x].dtype == 'O'] # type: object
num_col = [x for x in df.columns if x not in cat_col] # type: float64, int64

print(f' cat_col : {len(cat_col)} \n \
num col : {len(num_col)}')

 cat_col : 6 
 num col : 14


## Replace Missing values

In [184]:
# replace Missing values(Unknown, nan) with the most frequent values
# unique values
for col in cat_col:
  print(f'{col}: {df[col].unique()}') # We found 'Unknown'

print(f'\nnumber of missing values : {len(df[df == "Unknown"].dropna(how="all", axis=0))}\n') # 3046
print()
# the most frequent values
most_frequent_value = df[cat_col].value_counts().idxmax()
print(f'most_frequent_values : {most_frequent_value}\n')


Attrition_Flag: ['Existing Customer' 'Attrited Customer']
Gender: ['M' 'F']
Education_Level: ['High School' 'Graduate' 'Uneducated' 'Unknown' 'College' 'Post-Graduate'
 'Doctorate']
Marital_Status: ['Married' 'Single' 'Unknown' 'Divorced']
Income_Category: ['$60K - $80K' 'Less than $40K' '$80K - $120K' '$40K - $60K' '$120K +'
 'Unknown']
Card_Category: ['Blue' 'Gold' 'Silver' 'Platinum']

number of missing values : 3046


most_frequent_values : ('Existing Customer', 'F', 'Graduate', 'Married', 'Less than $40K', 'Blue')



In [185]:
# replacement
for i in range(len(cat_col)):
  idx = df[df[cat_col].iloc[:, i] == 'Unknown'].index
  df.loc[idx, cat_col[i]] = most_frequent_value[i]

for col in cat_col:
  print(f'{col}: {df[col].unique()}') # 결과 확인

Attrition_Flag: ['Existing Customer' 'Attrited Customer']
Gender: ['M' 'F']
Education_Level: ['High School' 'Graduate' 'Uneducated' 'College' 'Post-Graduate'
 'Doctorate']
Marital_Status: ['Married' 'Single' 'Divorced']
Income_Category: ['$60K - $80K' 'Less than $40K' '$80K - $120K' '$40K - $60K' '$120K +']
Card_Category: ['Blue' 'Gold' 'Silver' 'Platinum']


## Train/Test data Split

In [193]:
train, test = train_test_split(df, test_size = 0.3, random_state = 42)

In [194]:
print(f' train size : {len(train)} \n test size : {len(test)}')

 train size : 7088 
 test size : 3039


## Standardization

In [199]:
# Standardization
scaler = StandardScaler()
train[num_col] = scaler.fit_transform(train[num_col])
test[num_col] = scaler.transform(test[num_col])

In [200]:
train[num_col].describe().T # 결과 확인, (min, max) 값을 통해서 outliers 존재 확인

,count,mean,std,min,25%,50%,75%,max
Customer_Age,7088.0,-3.879515e-16,1.000071,-2.519352,-0.657692,-0.037139,0.707525,2.693296
Dependent_count,7088.0,-1.513713e-16,1.000071,-1.811368,-1.032871,-0.254374,0.524124,2.081118
Months_on_book,7088.0,3.769245e-16,1.000071,-2.848106,-0.613245,0.007550,0.504185,2.490728
Total_Relationship_Count,7088.0,8.671268e-17,1.000071,-1.798211,-0.514072,0.127997,0.770067,1.412136
Months_Inactive_12_mon,7088.0,-4.811802e-17,1.000071,-2.307381,-0.337527,-0.337527,0.647401,3.602182
Contacts_Count_12_mon,7088.0,1.318233e-16,1.000071,-2.228355,-0.409541,-0.409541,0.499866,3.228087
Credit_Limit,7088.0,2.506147e-17,1.000071,-0.794501,-0.671725,-0.450779,0.283363,2.869310
Total_Revolving_Bal,7088.0,9.523358e-17,1.000071,-1.426588,-0.997632,0.143078,0.762101,1.664842
Avg_Open_To_Buy,7088.0,-8.470776e-17,1.000071,-0.825336,-0.677514,-0.438144,0.278302,2.999812
Total_Amt_Chng_Q4_Q1,7088.0,-7.613674e-16,1.000071,-3.523910,-0.593569,-0.106724,0.455466,8.879037


## Remove Outliers

In [201]:
# Remove rows with Ouliers by Z-score (threshold: 3)
# apply only for train dataset

outliers =train[(train[num_col] > 3) | (train[num_col] < -3)].dropna(how='all', axis=0)
print(f'number of outliers : {len(outliers)}')

train = train.drop(index = outliers.index)
train.describe().T # 결과 확인

number of outliers : 537


,count,mean,std,min,25%,50%,75%,max
Customer_Age,6551.0,0.002760,0.992861,-2.519352,-0.657692,-0.037139,0.707525,2.693296
Dependent_count,6551.0,0.002788,0.997295,-1.811368,-1.032871,-0.254374,0.524124,2.081118
Months_on_book,6551.0,0.001902,0.992838,-2.848106,-0.613245,0.007550,0.504185,2.490728
Total_Relationship_Count,6551.0,0.026164,0.996530,-1.798211,-0.514072,0.127997,0.770067,1.412136
Months_Inactive_12_mon,6551.0,-0.044649,0.918537,-2.307381,-0.337527,-0.337527,0.647401,2.617255
Contacts_Count_12_mon,6551.0,-0.013628,0.974445,-2.228355,-0.409541,-0.409541,0.499866,2.318680
Credit_Limit,6551.0,-0.022975,0.987467,-0.794501,-0.677734,-0.469111,0.242159,2.869310
Total_Revolving_Bal,6551.0,-0.010407,1.000441,-1.426588,-1.136114,0.133252,0.751047,1.664842
Avg_Open_To_Buy,6551.0,-0.022050,0.987779,-0.825336,-0.685117,-0.455489,0.234939,2.999812
Total_Amt_Chng_Q4_Q1,6551.0,-0.073426,0.844331,-2.967516,-0.621388,-0.129907,0.412577,2.985899


In [203]:
y_train = train['Attrition_Flag']
X_train = train.drop(columns=['Attrition_Flag'])

y_test = test['Attrition_Flag']
X_test = test.drop(columns=['Attrition_Flag'])

## One-Hot Encoding

In [204]:
cat_col = [x for x in X_train.columns if X_train[x].dtype == 'O']
X_train = pd.get_dummies(X_train, columns = cat_col)
X_test = pd.get_dummies(X_test, columns = cat_col)

In [205]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6551 entries, 415 to 7270
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Customer_Age                    6551 non-null   float64
 1   Dependent_count                 6551 non-null   float64
 2   Months_on_book                  6551 non-null   float64
 3   Total_Relationship_Count        6551 non-null   float64
 4   Months_Inactive_12_mon          6551 non-null   float64
 5   Contacts_Count_12_mon           6551 non-null   float64
 6   Credit_Limit                    6551 non-null   float64
 7   Total_Revolving_Bal             6551 non-null   float64
 8   Avg_Open_To_Buy                 6551 non-null   float64
 9   Total_Amt_Chng_Q4_Q1            6551 non-null   float64
 10  Total_Trans_Amt                 6551 non-null   float64
 11  Total_Trans_Ct                  6551 non-null   float64
 12  Total_Ct_Chng_Q4_Q1             

# Modeling

## Bagging
### Random Forest

In [206]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)

In [214]:
print(f'random forest accuracy : \n{classification_report(y_test,pred_rf)}')

random forest accuracy : 
                   precision    recall  f1-score   support

Attrited Customer       0.93      0.75      0.83       496
Existing Customer       0.95      0.99      0.97      2543

         accuracy                           0.95      3039
        macro avg       0.94      0.87      0.90      3039
     weighted avg       0.95      0.95      0.95      3039



## Boosting
### Gradient Boosting Classifier

In [216]:
gbm = GradientBoostingClassifier()
gbm.fit(X_train, y_train)
pred_gbm = gbm.predict(X_test)

In [219]:
print(f'gradient boosting classifier accuracy: \n{classification_report(y_test, pred_gbm)}')

gradient boosting classifier accuracy: 
                   precision    recall  f1-score   support

Attrited Customer       0.93      0.84      0.88       496
Existing Customer       0.97      0.99      0.98      2543

         accuracy                           0.96      3039
        macro avg       0.95      0.92      0.93      3039
     weighted avg       0.96      0.96      0.96      3039



# The prediction performace of Gradient Boosting machine was superior to that of the Random forest.
